In [ ]:
#!pip install --force-reinstall sql_sidekick-0.1.9-py3-none-any.whl

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sidekick import prompter

In [4]:
prompter.__version__

'0.1.9'

In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
from sidekick.prompter import db_setup, ask
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir, list_db_dialects

In [7]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [8]:
list_db_dialects()

['bigquery',
 'clickhouse',
 'duckdb',
 'hive',
 'mysql',
 'oracle',
 'postgres',
 'presto',
 'redshift',
 'snowflake',
 'spark',
 'spark2',
 'sqlite',
 'starrocks',
 'tableau',
 'trino',
 'tsql',
 'databricks',
 'drill',
 'teradata']

In [17]:
# env variables
import os

os.environ['DATABRICKS_HOST'] = ""
os.environ['DATABRICKS_CLUSTER_ID'] = ""
os.environ['DATABRICKS_TOKEN'] = ""
os.environ['OPENAI_API_KEY'] = "" # Needed only for openAI models

os.environ['H2O_BASE_MODEL_URL'] = ''
os.environ['H2O_BASE_MODEL_API_KEY'] = ""
os.environ['RECOMMENDATION_MODEL_REMOTE_URL'] = ""
os.environ['RECOMMENDATION_MODEL_API_KEY'] = ""

In [18]:
# Scenario 1
# 1. New dataset for QnA
# 2. Generate SQL syntax, but no access to Databricks cluster for runtime validation.

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
# Download dataset --> https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
data_path = "./demo/sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_eda_demo"

table_schema, table_info_path = generate_schema(data_path=data_path, output_path=f"{cache_path}/{table_name}_table_info.jsonl")
 
# Set add_sample=False if no need to add rows to the table
response , err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path,
                add_sample=True
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432
Database already exists!
Table name: sleep_health_eda_demo_9


2024-01-18 09:59:32.837 | DEBUG    | sidekick.db_config:_extract_schema_info:163 - Using schema information from: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_9_table_info.jsonl
2024-01-18 09:59:32.843 | DEBUG    | sidekick.db_config:create_table:187 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE
2024-01-18 09:59:32.847 | INFO     | sidekick.db_config:create_table:199 - Table created: sleep_health_eda_demo_9


Checked table sleep_health_eda_demo_9 exists in the DB.


2024-01-18 09:59:32.853 | DEBUG    | sidekick.db_config:add_samples:223 - Adding sample values to table: /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv
2024-01-18 09:59:32.861 | DEBUG    | sidekick.db_config:add_samples:229 - Inserting chunk: 0
2024-01-18 09:59:33.000 | INFO     | sidekick.db_config:add_samples:234 - Data inserted into table: sleep_health_eda_demo_9
2024-01-18 09:59:33.003 | INFO     | sidekick.db_config:add_samples:239 - Number of rows inserted: 2618


Created a Database querydb. Inserted sample values from /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv into table sleep_health_eda_demo_9, please ask questions!


In [10]:
table_info_path

'/home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_9_table_info.jsonl'

In [11]:
table_schema

[{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Gender',
  'Column Type': 'TEXT',
  'Sample Values': ['Male', 'Female']},
 {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Occupation',
  'Column Type': 'TEXT',
  'Sample Values': ['Engineer',
   'Salesperson',
   'Sales Representative',
   'Nurse',
   'Accountant',
   'Teacher',
   'Software Engineer',
   'Lawyer',
   'Manager',
   'Doctor']},
 {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
 {'Column Name': 'BMI_Category',
  'Column Type': 'TEXT',
  'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
 {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
 {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
 {'Column Name': 'Daily_Steps', 'Column Type': 'NUMERI

In [12]:
# One could also decide to pass pre-configured table schema via for table creation
# Schema format:

# [{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Gender',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Male', 'Female']},
#  {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Occupation',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Accountant',
#    'Manager',
#    'Sales Representative',
#    'Salesperson',
#    'Lawyer',
#    'Nurse',
#    'Teacher',
#    'Software Engineer',
#    'Doctor',
#    'Scientist']},
#  {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'BMI_Category',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
#  {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
#  {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Daily_Steps', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Sleep_Disorder',
#   'Column Type': 'TEXT',
#   'Sample Values': ['None', 'Sleep Apnea', 'Insomnia']}]




# response, err = db_setup(
#                 db_name=DB_NAME,
#                 hostname=HOST_NAME,
#                 user_name=USER_NAME,
#                 password=PASSWORD,
#                 port=PORT,
#                 table_name=table_name,
#                 table_schema = r,
#                 local_base_path=base_path,
#                 add_sample=False
#             )

In [19]:
def query(question: str, table_name, table_info_path: str, sample_qna_path: str, regenerate=False, regenerate_with_options=False):
    """Asks question and returns SQL."""
    base_path = "./"
    # self_correction is enabled by default, set to False if not needed.
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        db_dialect="databricks",
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder-34b-alpha",
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path,
    )
    _tmp = res[0][1].split("```")[1].replace("sql", '')
    sql_str = ' '.join(_tmp.split())
    return sql_str

In [23]:
res = query("What is the average sleep duration for each gender?", table_name=table_name, 
            table_info_path=table_info_path, sample_qna_path=None, regenerate=False)

2024-01-18 10:03:24.958 | INFO     | sidekick.prompter:ask:490 - Table in use: ['sleep_health_eda_demo_9']
2024-01-18 10:03:24.959 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: databricks
2024-01-18 10:03:24.960 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-18 10:03:24.961 | INFO     | sidekick.prompter:ask:525 - Question: What is the average sleep duration for each gender?
2024-01-18 10:03:24.962 | DEBUG    | sidekick.prompter:ask:543 - Table info path: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_9_table_info.jsonl
2024-01-18 10:03:24.963 | INFO     | sidekick.utils:is_resource_low:347 - Number of GPUs: 1
2024-01-18 10:03:24.964 | DEBUG    | sidekick.utils:is_resource_low:351 - Information on device: 0
2024-01-18 10:03:24.965 | INFO     | sidekick.utils:is_resource_low:354 - Total Memory: 23GB
2024-01-18 10:03:24.966 | INFO     | sidekick.utils:is_resource_low:355 - Free GPU memory: 22GB
2024-01-18 10:0

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /home/pramit/testing/q/apps/system/sql-sidekick/models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
Load pretrained SentenceTransformer: /home/pramit/testing/q/apps/system/sql-sidekick/models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/


2024-01-18 10:03:25.148 | INFO     | sidekick.utils:_check_file_info:461 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_9_table_info.jsonl
2024-01-18 10:03:25.149 | INFO     | sidekick.prompter:ask:593 - Computing user request ...
2024-01-18 10:03:25.222 | DEBUG    | sidekick.utils:semantic_search:147 - Input questions: # query: what is the average sleep duration for each gender?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-18 10:03:25.380 | DEBUG    | sidekick.utils:semantic_search:152 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-18 10:03:25.412 | DEBUG    | sidekick.utils:semantic_search:160 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2024-01-18 10:03:25.414 | DEBUG    | sidekick.utils:semantic_search:160 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2024-01-18 10:03:25.416 | DEBUG    | sidekick.utils:semantic_search:160 - Similarity score for: detailed summary: 0.8346069603076574
2024-01-18 10:03:25.417 | DEBUG    | sidekick.utils:semantic_search:160 - Similarity score for: summary: 0.8394152180082535
2024-01-18 10:03:25.418 | DEBUG    | sidekick.utils:semantic_search:176 - Sorted context: []
2024-01-18 10:03:25.419 | DEBUG    | sidekick.query:generate_sql:469 - Filter Context: []
2024-01-18 10:03:25.420 | INFO     | sidekick.query:generate_sql:477 - Filtering Question/Query pairs ...
2024-01-18 10:03:25.421 | INFO     | sidekick.query:generate_sql:479 - Number of context queries fou

INFO:httpx:HTTP Request: POST http://38.128.233.247:5000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://38.128.233.247:5000/v1/chat/completions "HTTP/1.1 200 OK"


2024-01-18 10:03:28.779 | DEBUG    | sidekick.query:generate_sql:617 - Generated tokens: Gender, AVG(Sleep_Duration) AS 'Average Sleep Duration' FROM [/INST]sleep_health_eda_demo_9 GROUP BY [/INST]Gender ORDER BY [/INST]Gender NULLS LAST;[/INST]
2024-01-18 10:03:28.784 | INFO     | sidekick.prompter:ask:598 - Input query: What is the average sleep duration for each gender?
2024-01-18 10:03:28.786 | INFO     | sidekick.prompter:ask:599 - Generated response:

SELECT `Gender`, AVG(`Sleep_Duration`) AS `Average Sleep Duration` FROM `sleep_health_eda_demo_9` GROUP BY `Gender` ORDER BY `Gender` NULLS LAST LIMIT 100
2024-01-18 10:03:28.793 | INFO     | sidekick.prompter:ask:625 - Alternate responses:

[]


Exiting...


In [24]:
res

'SELECT `Gender`, AVG(`Sleep_Duration`) AS `Average Sleep Duration` FROM `sleep_health_eda_demo_9` GROUP BY `Gender` ORDER BY `Gender` NULLS LAST LIMIT 100'

In [ ]:
# Scenario 2
# 1. Connect to pre-configured Catalog/Schema on Databricks cluster
# 2. Retrieve information
# 3. Generate SQL syntax, execute against configured DB for the final result

In [27]:
# Retrieve information
from sidekick.db_config import DBConfig

DBConfig.dialect = "databricks"
# Using a demo dataset from Databricks Catalog
config_args = {
    "catalog": "samples",
    "schema": "nyctaxi",
    "cluster_id": os.environ.get("DATABRICKS_CLUSTER_ID")
}
table_name = "trips" # sample table related to NYC Taxi dataset
DBConfig.table_name = table_name
column_info, table_info_path = DBConfig.get_column_info(output_path=f"{cache_path}/{table_name}_table_info.jsonl", **config_args)

INFO:databricks.sql.client:Successfully opened session 015a850c-fecc-450d-88e2-5d809cb91ead
Successfully opened session 015a850c-fecc-450d-88e2-5d809cb91ead


In [29]:
input_q = "Compute average trip distance"
result, _, error = ask(
        question=input_q,
        table_info_path=table_info_path,
        sample_queries_path=None,
        table_name=table_name,
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder-34b-alpha",
        db_dialect="databricks",
        execute_db_dialect="databricks",
        is_regenerate=False,
        is_regen_with_options=False,
        execute_query=True,
        local_base_path=base_path,
        debug_mode=False,
        guardrails=True,
        self_correction=True
    )

2024-01-18 10:28:11.245 | INFO     | sidekick.prompter:ask:490 - Table in use: ['trips']
2024-01-18 10:28:11.246 | INFO     | sidekick.prompter:ask:491 - SQL dialect for generation: databricks
2024-01-18 10:28:11.247 | INFO     | sidekick.prompter:ask:524 - Setting context...
2024-01-18 10:28:11.248 | INFO     | sidekick.prompter:ask:525 - Question: Compute average trip distance
2024-01-18 10:28:11.249 | DEBUG    | sidekick.prompter:ask:543 - Table info path: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/trips_table_info.jsonl
2024-01-18 10:28:11.250 | INFO     | sidekick.utils:is_resource_low:347 - Number of GPUs: 1
2024-01-18 10:28:11.251 | DEBUG    | sidekick.utils:is_resource_low:351 - Information on device: 0
2024-01-18 10:28:11.252 | INFO     | sidekick.utils:is_resource_low:354 - Total Memory: 23GB
2024-01-18 10:28:11.253 | INFO     | sidekick.utils:is_resource_low:355 - Free GPU memory: 22GB
2024-01-18 10:28:11.254 | DEBUG    | sidekick.utils:load_embedding_model:

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /home/pramit/testing/q/apps/system/sql-sidekick/models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
Load pretrained SentenceTransformer: /home/pramit/testing/q/apps/system/sql-sidekick/models/sentence_transformers/models--BAAI--bge-base-en/snapshots/17d124a4b773c4c9248ca816b0b0901e3c49a243/
INFO:databricks.sql.client:Successfully opened session f64847ca-f3ed-4208-87fb-abec2b615ff1
Successfully opened session f64847ca-f3ed-4208-87fb-abec2b615ff1


2024-01-18 10:28:16.516 | INFO     | sidekick.utils:_check_file_info:461 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/trips_table_info.jsonl
2024-01-18 10:28:16.521 | INFO     | sidekick.prompter:ask:593 - Computing user request ...
2024-01-18 10:28:18.094 | DEBUG    | sidekick.utils:semantic_search:147 - Input questions: # query: compute average trip distance


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-18 10:28:18.274 | DEBUG    | sidekick.utils:semantic_search:152 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


...

INFO:databricks.sql.client:Closing session bed8459d-89c8-423b-a057-7d8c20db96f4
Closing session bed8459d-89c8-423b-a057-7d8c20db96f4


2024-01-18 10:28:49.659 | DEBUG    | sidekick.prompter:ask:670 - Attempt: 2


Please install the correct version of H2OGPTE with `pip install h2ogpte==1.3.0-dev3`.
You can enable strict version checking by passing strict_version_check=True.
INFO:databricks.sql.client:Successfully opened session 9bbedfe5-9a09-41d2-9eea-3c9bef34d352
Successfully opened session 9bbedfe5-9a09-41d2-9eea-3c9bef34d352


2024-01-18 10:29:03.041 | DEBUG    | sidekick.db_config:execute_query:268 - Executing query:
 SELECT AVG(`t`.`trip_distance`) AS `average_trip_distance` FROM `trips` AS `t` LIMIT 100


INFO:databricks.sql.client:Closing session 9bbedfe5-9a09-41d2-9eea-3c9bef34d352
Closing session 9bbedfe5-9a09-41d2-9eea-3c9bef34d352
The query results are:
 [(2.8528291993434256,)]
Exiting...


In [ ]:
?ask